# 대규모 그래프에서의 노드 분류를 위한 GNN의 확률적(Stochastic) 학습

이번 튜토리얼에서는, OGB에서 제공하는 Amazon Copurchase Network 데이터로 노드 분류를 수행하는 멀티 레이어 GraphSAGE를 학습하는 방법을 배워 봅니다.  
이 데이터셋은 240만 노드와 6,100만 엣지를 포함하며, 따라서 단독 GPU에 모두 올려 사용할 수 없습니다.  

이번 튜토리얼의 컨텐츠는 다음을 포함합니다.  

* CSV 형식과 같은 형식으로 저장된 자기만의 데이터로 DGL 그래프 만들기
* GNN 모델을 1개의 머신으로, 1개의 GPU만을 사용해, 어떤 크기의 그래프든 학습하기

## 데이터셋 로드하기


OGB에서 제공하는 파이썬 패키지를 직접 사용할 수 있지만, 설명을 위해 수동으로 데이터셋을 다운받고, 내용물을 확인하고, 오직 `numpy`로만 처리하겠습니다.  

In [1]:
# Check cuda version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
pip install dgl-cu110

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wget https://snap.stanford.edu/ogb/data/nodeproppred/products.zip

--2023-03-26 13:22:42--  https://snap.stanford.edu/ogb/data/nodeproppred/products.zip
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1480993786 (1.4G) [application/zip]
Saving to: ‘products.zip.1’

products.zip.1      100%[===================>]   1.38G  29.7MB/s    in 42s     

2023-03-26 13:23:24 (33.9 MB/s) - ‘products.zip.1’ saved [1480993786/1480993786]



In [4]:
!unzip -o products.zip

Archive:  products.zip
  inflating: products/split/sales_ranking/test.csv.gz  
  inflating: products/split/sales_ranking/train.csv.gz  
  inflating: products/split/sales_ranking/valid.csv.gz  
  inflating: products/raw/node-label.csv.gz  
 extracting: products/raw/num-node-list.csv.gz  
 extracting: products/raw/num-edge-list.csv.gz  
  inflating: products/raw/node-feat.csv.gz  
  inflating: products/raw/edge.csv.gz  
  inflating: products/mapping/README.md  
 extracting: products/mapping/labelidx2productcategory.csv.gz  
  inflating: products/mapping/nodeidx2asin.csv.gz  
  inflating: products/RELEASE_v1.txt  


이 데이터셋에는 다음 파일들이 포함되어 있습니다:

* `products/raw/edge.csv` (source-destination pairs)
* `products/raw/node-feat.csv` (node features)
* `products/raw/node-label.csv` (node labels)
* `products/raw/num-edge-list.csv` (number of edges)
* `products/raw/num-node-list.csv` (number of nodes)

이 중에서 처음 3개의 csv 파일만을 사용하겠습니다.  

추가로, 이 데이터셋에는 학습-검증-테스트셋 분할을 정의하는 파일들이 `products/split/sales_ranking` 디렉터리에 포함되어 있습니다.  
`train.csv`, `valid.csv` 그리고 `test.csv` 모두는 학습/검증/테스트셋의 노드 ID가 한 줄에 하나씩 포함된 텍스트 파일입니다.  


<div class="alert alert-info">
    <b>주의:</b> 노드 ID는 0부터 (전체 노드의 숫자-1)까지 이어지는 정수여야 합니다. 만약 노드 ID가 연속되지 않거나 0부터 시작된다면(가령, 100000부터 시작한다던지.),   
    라벨을 직접 다시 달아주어야 합니다. 판다스 데이터프레임의 <code>astype</code> 메서드는 ID들의 타입을 <code>"category"</code>로 바꾸어 줌으로써 간편하게 재라벨링할 수 있습니다.  
</div>

In [5]:
import pandas as pd
edges = pd.read_csv('products/raw/edge.csv.gz', header=None).values
node_features = pd.read_csv('products/raw/node-feat.csv.gz', header=None).values
node_labels = pd.read_csv('products/raw/node-label.csv.gz', header=None).values[:, 0]

# pd.read_csv는 칼럼 1개짜리 데이터프레임을 호출하므로, 1차원 배열로 만들어줍니다.
train_nids = pd.read_csv('products/split/sales_ranking/train.csv.gz', header=None).values[:, 0]
valid_nids = pd.read_csv('products/split/sales_ranking/valid.csv.gz', header=None).values[:, 0]
test_nids = pd.read_csv('products/split/sales_ranking/test.csv.gz', header=None).values[:, 0]

아래와 같이 그래프를 구축합니다.

In [6]:
import dgl
import torch

graph = dgl.graph((edges[:, 0], edges[:, 1]))
node_features = torch.FloatTensor(node_features)
node_labels = torch.LongTensor(node_labels)

# 그래프와 피처, 그리고 학습-검증-테스트 분할 정보를 이후의 튜토리얼에서 사용하기 위해 분할합니다.

import pickle
with open('data.pkl', 'wb') as f:
    pickle.dump((graph, node_features, node_labels, train_nids, valid_nids, test_nids), f)

Using backend: pytorch


In [7]:
# 저장한 파일로부터 그래프를 다시 호출합니다.

import dgl
import torch
import numpy as np
import pickle
with open('data.pkl', 'rb') as f:
    graph, node_features, node_labels, train_nids, valid_nids, test_nids = pickle.load(f)

그래프, 피처, 라벨의 사이즈를 아래와 같이 확인할 수 있습니다.

In [8]:
print('그래프 정보')
print(graph)
print('노드 피처의 shape:', node_features.shape)
print('노드 라벨의 shape:', node_labels.shape)

num_features = node_features.shape[1]
num_classes = (node_labels.max() + 1).item()
print('클래스의 수:', num_classes)

그래프 정보
Graph(num_nodes=2449029, num_edges=61859140,
      ndata_schemes={}
      edata_schemes={})
노드 피처의 shape: torch.Size([2449029, 100])
노드 라벨의 shape: torch.Size([2449029])
클래스의 수: 47


## 이웃 샘플링으로 데이터 로더 정의하기

### 이웃 샘플링 개요


message passing의 수식은 일반적으로 아래의 형태를 따릅니다.

$$
\begin{gathered}
  \boldsymbol{a}_v^{(l)} = \rho^{(l)} \left(
    \left\lbrace
      \boldsymbol{h}_u^{(l-1)} : u \in \mathcal{N} \left( v \right)
    \right\rbrace
  \right)
\\
  \boldsymbol{h}_v^{(l)} = \phi^{(l)} \left(
    \boldsymbol{h}_v^{(l-1)}, \boldsymbol{a}_v^{(l)}
  \right)
\end{gathered}
$$

v라는 노드가 있고 u가 neighbor, undirected 그래프 기준으로 N(v)가 v의neighbor 집합이었는데, directed graph 형태까지 고려한 일반화된 그래프 형태
u에 대한 l-1번째의 hidden 표현을 aggregation하는 것을 첫줄처럼 표현하고
주변으로부터취합한 정보와 자기 자신으로부터 표현을 가지고 update 함수 적용한 것이 l번째 layer의 은닉 표현이 된다.


$\rho^{(l)}$ 와 $\phi^{(l)}$는 파라미터화된 함수이고,  $\mathcal{N}(v)$은 그래프 $\mathcal{G}$ 내에 있는 $v$의 **predecessors**(혹은 *이웃*이라고도 불립니다.)를 나타냅니다.

$$
\mathcal{N} \left( v \right) = \left\lbrace
  s \left( e \right) : e \in \mathbb{E}, t \left( e \right) = v
\right\rbrace
$$

예를 들어, 아래의 빨간 노드를 message passing을 통해 업데이트 하기 위해서는


![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/1.png?raw=1)

그 이웃의 노드 피처를 통합할 필요가 있습니다. 아래의 녹색 노드를 보세요.

![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/2.png?raw=1)

한 노드의 출력을 계산할 때 다중 레이어의 message passing이 어떻게 작동하는지 살펴 봅시다.  
아래의 내용은, GNN이 seed 노드로 간주하여 계산하는 결과값을 만들어 내는 노드에 대한 설명입니다.  


2-레이어 GNN으로 seed 노드 8의 출력값을 계산하는 상황을 생각해 봅시다. 아래의 그래프에서 빨간색으로 칠해져 있습니다.

![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/seed.png?raw=1)

수식은 다음과 같습니다.

$$
\begin{gathered}
  \boldsymbol{a}_8^{(2)} = \rho^{(2)} \left(
    \left\lbrace
      \boldsymbol{h}_u^{(1)} : u \in \mathcal{N} \left( 8 \right)
    \right\rbrace
  \right) = \rho^{(2)} \left(
    \left\lbrace
      \boldsymbol{h}_4^{(1)}, \boldsymbol{h}_5^{(1)},
      \boldsymbol{h}_7^{(1)}, \boldsymbol{h}_{11}^{(1)}
    \right\rbrace
  \right)
\\
  \boldsymbol{h}_8^{(2)} = \phi^{(2)} \left(
    \boldsymbol{h}_8^{(1)}, \boldsymbol{a}_8^{(2)}
  \right)
\end{gathered}
$$


수식에서 볼 수 있듯이, $\boldsymbol{h}_8^{(2)}$ 을 계산하기 위해서는, 4,5,7,11번(녹색으로 칠해진) 노드에서 message를 아래의 시각화된 엣지를 따라 받아야 합니다.

![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/3.png?raw=1)

   $\boldsymbol{h}_\cdot^{(1)}$의 값들은 첫번째 GNN 레이어로부터 나온 출력값입니다.  
   이러한 값들을 빨간색, 녹색 노드에서 계산하기 위해서는, 아래 시각화된 엣지들에 대한 message passing도 수행할 필요가 있습니다.  


![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/4.png?raw=1)

따라서, 빨간 노드의 2-레이어 GNN 표현을 계산하기 위해서는, 빨간 노드의 입력 피처값 뿐만 아니라 녹색, 노란색 노드의 입력 피처값도 필요합니다.  
이 레이어에서 빨간 노드의 이웃들을 다시 취해 준다는 사실에 주목해 주세요.  

연산 의존성(computation dependency)을 결정하는 이 절차는 message 통합의 반대 방향에서 이루어 진다는 점에 주목해 주세요.  
즉, 출력 층에 가장 가까운 레이어부터 시작해 입력까지 거꾸로 작동한다는 말이지요.

많지 않은 노드의 표현을 계산하는 작업이 종종 훨씬 더 큰 수의 노드의 입력 피처를 필요로 한다는 점도 알 수 있습니다.  
message 통합을 위해 모든 이웃을 취해주는 일은 보통 너무 큰 비용이 들어갑니다. 필요한 노드를 감안하면 그래프의 큰 부분을 포함하기 때문이죠.  

이웃 샘플링은 message 통합 수행 시 이웃의 무작위적인 부분집합을 선택함으로써 이런 문제를 해결합니다.  
예를 들어, $\boldsymbol{h}_8^{(1)}$를 계산하기 위해, 2개의 이웃 노드를 골라 통합할 수 있습니다.

![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/5.png?raw=1)

비슷한 방식으로, 빨간색 그리고 녹색 노드의 첫번째 레이어 표현을 계산하기 위해, 각 노드에서 2개의 이웃 노드만을 취하는 이웃 샘플링을 수행할 수 있습니다. 빨간 노드의 이웃 노드들을 이번 레이어에서 또 취해주어야 한다는 점에 주목해 주세요.

![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/6.png?raw=1)

이러한 방식으로 입력 피처를 위해 필요한 노드가 줄어들었음을 알 수 있습니다.

## DGL에서 이웃 샘플러와 데이터로더 정의하기

DGL은 데이터셋을 미니배치로 반복하며 이러한 연산 의존성(computation dependencies)을 생성하는 유용한 툴을 제공합니다.  
노드 분류 작업에서, `dgl.dataloading.NodeDataLoader`를 사용해 데이터셋에 걸쳐 반복할 수 있으며,  
`dgl.dataloading.MultiLayerNeighborSampler`를 사용하여 이웃 샘플링을 통한 다중 레이어 GNN에서의 노드의 연산 의존성을 생성할 수 있습니다.  

`dgl.dataloading.NodeDataLoader`의 문법은 PyTorch의 `DataLoader`와 거의 유사한데,  
이에 더해 연산 의존성을 생성할 그래프와 반복할 노드 ID 집합, 그리고 여러분이 정의한 이웃 샘플러가 필요합니다.  

이웃 샘플링을 사용한 3-레이어 GraphSAGE를 학습시켜 봅시다.  
여기서 각 노드는 각 레이어마다 4개의 이웃 노드로부터 message를 받습니다.  
data loader와 이웃 샘플러를 정의하는 코드는 아래와 같이 생겼습니다.

In [9]:
sampler = dgl.dataloading.MultiLayerNeighborSampler([4, 4, 4])
train_dataloader = dgl.dataloading.NodeDataLoader(
    graph, train_nids, sampler,
    batch_size=1024,
    shuffle=True,
    drop_last=False,
    num_workers=0
)

우리가 만든 data loader에 걸쳐 반복할 수 있겠죠. 그 결과를 확인해 봅시다.

In [10]:
example_minibatch = next(iter(train_dataloader))
print(example_minibatch)

[tensor([ 97584, 155798, 120577,  ...,  14215,  35609,  59280]), tensor([ 97584, 155798, 120577,  ..., 102220, 168573, 171706]), [Block(num_src_nodes=34972, num_dst_nodes=15837, num_edges=51644), Block(num_src_nodes=15837, num_dst_nodes=4617, num_edges=16117), Block(num_src_nodes=4617, num_dst_nodes=1024, num_edges=3726)]]


`NodeDataLoader`는 1회 iteration마다 3개의 item을 제공합니다.  

* 출력을 계산하기 위해 필요한 입력 피처를 가진 노드의 입력 노드 리스트  
* GNN 표현이 계산될 출력 노드 리스트
* 각 레이어의 연산 의존성 리스트


In [11]:
input_nodes, output_nodes, bipartites = example_minibatch
print("To compute {} nodes' output we need {} nodes' input features".format(len(output_nodes), len(input_nodes)))

To compute 1024 nodes' output we need 34972 nodes' input features


변수 `bipartites`는 각 레이어에서 어떻게 message가 통합되는지를 보여줍니다.  
이 이름이 암시하듯이, 이는 bipartite 그래프의 **리스트** 입니다.  
그런데 왜 DGL이 동질적(homogeneous) 그래프를 학습시키는 데 bipartite graph를 반환할까요? 

그 이유는 GNN 레이어에서 주어진 입력을 위한 노드의 수와 아웃풋을 위한 노드의 수가 다르기 때문입니다.  위의 예시를 다시 들어 설명하겠습니다.

![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/6.png?raw=1)

이 GNN 레이어는 노드 3개의 표현을 출력할 것입니다(2개의 녹색 노드, 그리고 1개의 빨간 노드) 그러나 입력을 위해서는 7개의 노드가 필요하죠(녹색 노드와 빨간 노드, 거기에 4개의 노란 노드까지).  
오직 bipartite 그래프만이 이런 계산을 묘사할 수 있을 것입니다.

![](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/bipartite.png?raw=1)

GNN의 미니배치 학습은 보통 이런 bipartite 그래프 상의 message passing을 포함합니다.

In [12]:
print(bipartites)

[Block(num_src_nodes=34972, num_dst_nodes=15837, num_edges=51644), Block(num_src_nodes=15837, num_dst_nodes=4617, num_edges=16117), Block(num_src_nodes=4617, num_dst_nodes=1024, num_edges=3726)]


## 모델 정의하기

모델은 아래처럼 쓰여질 수 있습니다.

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn as dglnn

class SAGE(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, n_layers):
        super().__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.layers = nn.ModuleList()
        self.layers.append(dglnn.SAGEConv(in_feats, n_hidden, 'mean'))
        for i in range(1, n_layers - 1):
            self.layers.append(dglnn.SAGEConv(n_hidden, n_hidden, 'mean'))
        self.layers.append(dglnn.SAGEConv(n_hidden, n_classes, 'mean'))
        
    def forward(self, bipartites, x):
        for l, (layer, bipartite) in enumerate(zip(self.layers, bipartites)):
            x = layer(bipartite, x)
            if l != self.n_layers - 1:
                x = F.relu(x)
        return x

여기서, 데이터 로더에 의해 생성된 한 쌍의 NN 모듈 레이어와 bipartite 그래프를 반복해 사용하고 있음을 볼 수 있습니다.

## 학습 루프 정의하기

아래의 내용은 모델을 초기화하고, optimizer를 정의합니다.

In [14]:
model = SAGE(num_features, 128, num_classes, 3).cuda()
opt = torch.optim.Adam(model.parameters())

모델 선택의 validation score를 계산할 때, 이 때 역시도 보통은 이웃 샘플링을 사용할 수 있습니다.   
이를 위해선, 다른 데이터 로더를 정의할 필요가 있습니다.

In [15]:
valid_dataloader = dgl.dataloading.NodeDataLoader(
    graph, valid_nids, sampler,
    batch_size=1024,
    shuffle=False,
    drop_last=False,
    num_workers=0
)

아래는 매 epoch마다 validation을 수행하는 학습 루프입니다.   
또한 가장 좋은 validation accuracy를 가진 모델을 파일로 저장해 줍니다.

In [16]:
import tqdm
import sklearn.metrics

best_accuracy = 0
best_model_path = 'model.pt'
for epoch in range(10):
    model.train()
    
    with tqdm.tqdm(train_dataloader) as tq:
        for step, (input_nodes, output_nodes, bipartites) in enumerate(tq):
            bipartites = [b.to(torch.device('cuda')) for b in bipartites]
            inputs = node_features[input_nodes].cuda()
            labels = node_labels[output_nodes].cuda()
            predictions = model(bipartites, inputs)

            loss = F.cross_entropy(predictions, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()

            accuracy = sklearn.metrics.accuracy_score(labels.cpu().numpy(), predictions.argmax(1).detach().cpu().numpy())
            
            tq.set_postfix({'loss': '%.03f' % loss.item(), 'acc': '%.03f' % accuracy}, refresh=False)
        
    model.eval()
    
    predictions = []
    labels = []
    with tqdm.tqdm(valid_dataloader) as tq, torch.no_grad():
        for input_nodes, output_nodes, bipartites in tq:
            bipartites = [b.to(torch.device('cuda')) for b in bipartites]
            inputs = node_features[input_nodes].cuda()
            labels.append(node_labels[output_nodes].numpy())
            predictions.append(model(bipartites, inputs).argmax(1).cpu().numpy())
        predictions = np.concatenate(predictions)
        labels = np.concatenate(labels)
        accuracy = sklearn.metrics.accuracy_score(labels, predictions)
        print('Epoch {} Validation Accuracy {}'.format(epoch, accuracy))
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), best_model_path)

100%|██████████| 39/39 [00:05<00:00,  7.54it/s]


Epoch 0 Validation Accuracy 0.8264883147267502


100%|██████████| 39/39 [00:04<00:00,  7.98it/s]


Epoch 1 Validation Accuracy 0.8526562062914833


100%|██████████| 39/39 [00:04<00:00,  8.24it/s]


...

Epoch 7 Validation Accuracy 0.8777051598301249


100%|██████████| 39/39 [00:05<00:00,  7.56it/s]


Epoch 8 Validation Accuracy 0.8793581364595784


100%|██████████| 39/39 [00:05<00:00,  7.58it/s]


Epoch 9 Validation Accuracy 0.8808839610406124


## 이웃 샘플링 없이 Offline에서 추론하기 


일반적으로 offline 추론에서는 이웃 샘플링에 의해 발생하는 무작위성을 제거하기 위해 전체 이웃에 대해 통합을 진행하는 것이 바람직합니다.   
하지만, 같은 방법을 학습 단계에서도 사용하는 것은 비효율적인데, 그 까닭은 너무 불필요한 연산이 많아지기 때문입니다.   
더욱이, 단순히 모든 이웃을 취해 이웃 샘플링을 수행하는 것은 종종 GPU 메모리를 모두 잡아먹을 수도 있는데, 이는 입력 피처를 위해 필요한 노드의 수가 GPU 메모리에 올려지기에 너무 클 수 있기 때문입니다.   


대신, 레이어마다 표현을 계산해주면 됩니다.  
즉, 먼저 모든 노드에 대해 첫번째 GNN 레이어의 출력 값을 계산하고,   
그 뒤 두번째 레이어의 출력 값을 모든 노드에 대해 계산하는 데 이 때 첫번째 GNN 레이어의 출력을 입력 값으로 사용하는 식입니다.   
이러한 방식은 학습 시에 사용된 것과는 다른 알고리즘이 됩니다.   


학습 중에는 노드에 걸쳐 돌아가는 외부 루프와, 레이어에 걸쳐 돌아가는 내부 루프가 있습니다.   
반대로, 추론 단계에서는 레이어에 걸쳐 돌아가는 외부 루프와 노드에 걸쳐 돌아가는 내부 루프가 있게 됩니다.  

만약 무작위성에 대해 크게 신경쓰지 않는다면, (가령 validation 상에서 모델을 선택하는 중이라던지)   
`dgl.dataloading.MultiLayerNeighborSampler`와 `dgl.dataloading.NodeDataLoader`를 사용해 offline 추론을 수행할 수 있습니다.   
이는 노드의 수가 적은 경우 evaluation을 수행하는 데 보통 더 빠르기 때문입니다.  

![Imgur](https://github.com/myeonghak/DGL-tutorial/blob/master/large_graph/assets/anim.gif?raw=1)

In [17]:
def inference(model, graph, input_features, batch_size):
    nodes = torch.arange(graph.number_of_nodes())
    
    sampler = dgl.dataloading.MultiLayerNeighborSampler([None])  # one layer at a time, taking all neighbors
    dataloader = dgl.dataloading.NodeDataLoader(
        graph, nodes, sampler,
        batch_size=batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=0)
    
    with torch.no_grad():
        for l, layer in enumerate(model.layers):
            # Allocate a buffer of output representations for every node
            # Note that the buffer is on CPU memory.
            output_features = torch.zeros(
                graph.number_of_nodes(), model.n_hidden if l != model.n_layers - 1 else model.n_classes)

            for input_nodes, output_nodes, bipartites in tqdm.tqdm(dataloader):
                bipartite = bipartites[0].to(torch.device('cuda'))

                x = input_features[input_nodes].cuda()

                # the following code is identical to the loop body in model.forward()
                x = layer(bipartite, x)
                if l != model.n_layers - 1:
                    x = F.relu(x)

                output_features[output_nodes] = x.cpu()
            input_features = output_features
    return output_features

아래의 코드는 이전에 저장된 파일에서부터 최적의 모델을 호출해 offline 추론을 수행합니다.  
그 뒤 테스트 셋에 대해 정확도를 계산합니다.

In [18]:
model.load_state_dict(torch.load(best_model_path))
all_predictions = inference(model, graph, node_features, 8192)

100%|██████████| 299/299 [00:42<00:00,  7.02it/s]


In [19]:
test_predictions = all_predictions[test_nids].argmax(1)
test_labels = node_labels[test_nids]
test_accuracy = sklearn.metrics.accuracy_score(test_predictions.numpy(), test_labels.numpy())
print('Test accuracy:', test_accuracy)

Test accuracy: 0.7291900784920277


## 결론

이 튜토리얼에서, 다중-레이어 GraphSAGE 모델을 이웃 샘플링을 통해 GPU에 맞지 않을 정도로 큰 데이터셋에 대해 학습하는 방법을 배웠습니다.   
지금 배운 이 방법은 어떤 사이즈의 그래프에도 확장 가능하며, 1개의 GPU를 가진 1개의 머신에서도 돌아갈 것입니다.


## 다음은 무엇인가요?

다음 튜토리얼은 똑같은 GraphSAGE 모델을 비지도학습적인 방식으로 link prediction 태스크에 학습해 봅니다.   
즉, 두 노드 사이에 엣지가 존재하는지 아닌지를 예측해 봅니다.